In [ ]:
import tweepy
import pandas as pd
import numpy as np
import yfinance as yf
from transformers import pipeline
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime, timedelta

# Step 1: Download Bitcoin Price Data
def download_btc_price(start_date, end_date):
    btc = yf.download("BTC-USD", start=start_date, end=end_date)
    return btc[["Close"]]

# Step 2: Download Live X Posts
def download_x_posts(api_key, api_secret, access_token, access_secret, query="#Bitcoin", count=500):
    auth = tweepy.OAuthHandler(api_key, api_secret)
    auth.set_access_token(access_token, access_secret)
    api = tweepy.API(auth)

    posts = tweepy.Cursor(api.search_tweets, q=query, lang="en", tweet_mode="extended").items(count)
    data = []
    for post in posts:
        try:
            text = post.full_text
            timestamp = post.created_at
            retweets = post.retweet_count
            data.append([timestamp, text, retweets])
        except AttributeError:
            continue

    return pd.DataFrame(data, columns=["timestamp", "text", "retweets"])

# Step 3: Sentiment Analysis
def analyze_sentiment(posts_df):
    sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased")
    posts_df["sentiment"] = posts_df["text"].apply(
        lambda x: sentiment_analyzer(x)[0]["score"] if sentiment_analyzer(x)[0]["label"] == "POSITIVE" else -sentiment_analyzer(x)[0]["score"]
    )
    return posts_df

# Step 4: Prepare Latest Data for Prediction
def prepare_latest_data(price_df, posts_df, scaler, seq_length=7):
    posts_df["date"] = posts_df["timestamp"].dt.date
    daily_sentiment = posts_df.groupby("date")["sentiment"].mean()
    data = pd.merge(price_df, daily_sentiment, left_index=True, right_index=True, how="inner")
    scaled_data = scaler.transform(data[["Close", "sentiment"]])
    last_sequence = scaled_data[-seq_length:].reshape(1, seq_length, 2)
    return last_sequence, data

# Main Execution
if __name__ == "__main__":
    # Define time range for latest data
    end_date = datetime(2025, 2, 25)
    start_date = end_date - timedelta(days=7)  # Last 7 days for prediction

    # Download latest Bitcoin price data
    price_df = download_btc_price(start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d"))

    # Replace with your X API credentials when account is restored
    api_key = "YOUR_API_KEY"
    api_secret = "YOUR_API_SECRET"
    access_token = "YOUR_ACCESS_TOKEN"
    access_secret = "YOUR_ACCESS_SECRET"
    posts_df = download_x_posts(api_key, api_secret, access_token, access_secret, query="#Bitcoin", count=500)

    # Process sentiment
    posts_df = analyze_sentiment(posts_df)

    # Load the trained model and scaler
    model = load_model("btc_model.h5")
    scaler = MinMaxScaler()  # Note: We'll fit it to historical data manually below

    # Fit scaler to historical data (you need to reuse the same scale)
    # For simplicity, we'll assume the CSV data range matches; in practice, save scaler in Script 1
    historical_price_df = download_btc_price("2025-01-26", "2025-02-25")  # Adjust to your CSV range
    historical_posts_df = pd.read_csv("bitcoin_tweets.csv")  # Load CSV again for scaler
    historical_posts_df["timestamp"] = pd.to_datetime(historical_posts_df["timestamp"])
    historical_posts_df = analyze_sentiment(historical_posts_df)
    _, _, scaler, _ = prepare_data(historical_price_df, historical_posts_df)

    # Prepare latest data
    last_sequence, data = prepare_latest_data(price_df, posts_df, scaler)

    # Predict
    prediction = model.predict(last_sequence)
    dummy = np.zeros((1, 2))
    dummy[0, 0] = prediction[0]
    predicted_price = scaler.inverse_transform(dummy)[0, 0]

    print(f"Predicted Bitcoin price for Feb 26, 2025 using live data: ${predicted_price:.2f}")

[*********************100%***********************]  1 of 1 completed


Unauthorized: 401 Unauthorized
89 - Invalid or expired token.